# # Day 3: Baseline Model for Nairobi House Price Prediction
# 
**Goal:** Build a simple linear regression model, evaluate its performance, and extract insights about price drivers.

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
from collections import Counter

import os

In [ ]:
# Load clean data (from Day 2)
df = pd.read_csv('../data/clean_listings.csv')
print(f"Clean data shape: {df.shape}")
df.head()

: 

In [ ]:
# ===================================================================================
#   2. Exploratory Analysis – Answer Key Questions

# ===================================================================================
#  1. Most Expensive Locations


# Median price by location (in millions KSh)
location_median = df.groupby('Location')['Price_Millions'].median().sort_values(ascending=False)
print("Top 10 most expensive locations (median price in M KSh):")
print(location_median.head(10))

# Plot
plt.figure(figsize=(10,6))
location_median.head(15).plot(kind='bar')
plt.title('Top 15 Locations by Median Price')
plt.ylabel('Median Price (Millions KSh)')
plt.tight_layout()
plt.savefig('../reports/location_median_price.png')
plt.show()

# ===================================================================================
#  2 How Strongly Does Size Affect Price?


corr = df['Size_SQM'].corr(df['Price_Millions'])
print(f"Correlation between Size (sqm) and Price: {corr:.2f}")

# Scatter plot with regression line
plt.figure(figsize=(10,6))
sns.regplot(x='Size_SQM', y='Price_Millions', data=df, scatter_kws={'alpha':0.5})
plt.title(f'Price vs Size (corr = {corr:.2f})')
plt.xlabel('Size (sqm)')
plt.ylabel('Price (Millions KSh)')
plt.savefig('../reports/price_vs_size.png')
plt.show()

# ===================================================================================
#  3 Which Amenities Increase Value Most?
# 
# To isolate the effect of amenities, run a regression that controls for size and location.


# Flatten all amenities to get the most common ones
all_amenities = []
for x in df['Amenities']:
    if isinstance(x, list):
        all_amenities.extend(x)
    elif isinstance(x, str):
        all_amenities.extend([a.strip() for a in x.split(',')])

top_10_amenities = [item for item, count in Counter(all_amenities).most_common(10)]
print("Top 10 amenities:", top_10_amenities)

# Create dummy variables for top amenities---- convert text amenities into binary columns
amenity_dummies = df['Amenities'].str.get_dummies(sep=', ')[top_10_amenities]

# Location dummies (drop first to avoid multicollinearity - 2 or more features having same info)
location_dummies = pd.get_dummies(df['Location'], drop_first=True)

# Combine features
X_amen = pd.concat([df[['Size_SQM']], location_dummies, amenity_dummies], axis=1).astype(float)
y_amen = df['Price_Millions'].astype(float)

# Drop rows with missing values (if any)
mask = X_amen.notnull().all(axis=1) & y_amen.notnull()
X_amen = X_amen[mask]
y_amen = y_amen[mask]

# Add constant for OLS
X_amen = sm.add_constant(X_amen)

# Fit model
model_amen = sm.OLS(y_amen, X_amen).fit()
print(model_amen.summary())

# Extract coefficients for the top amenities
amenity_effects = model_amen.params[top_10_amenities]
amenity_pvals = model_amen.pvalues[top_10_amenities]

results_df = pd.DataFrame({
    'Coefficient (M KSh)': amenity_effects,
    'P-value': amenity_pvals
}).sort_values('Coefficient (M KSh)', ascending=False)

print("\nAmenity Value Impact (Controlled for Size & Location):")
print(results_df)

# Plot with confidence intervals
std_err = model_amen.bse[top_10_amenities]
ci_lower = amenity_effects - 1.96 * std_err
ci_upper = amenity_effects + 1.96 * std_err

plt.figure(figsize=(10,6))
plt.barh(results_df.index, results_df['Coefficient (M KSh)'],
         xerr=[results_df['Coefficient (M KSh)'] - ci_lower, ci_upper - results_df['Coefficient (M KSh)']],
         color='skyblue', edgecolor='black')
plt.axvline(0, color='red', linestyle='--')
plt.xlabel('Estimated Price Impact (M KSh)')
plt.title('Top 10 Amenities: Price Impact (Controlled for Size & Location)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../reports/amenity_impact.png')
plt.show()

: 

In [ ]:
# ===================================================================================
#   3. Prepare Features for Baseline Model
# 
# use:
# - `Size_SQM`
# - `Bedrooms_Num`
# - `Bathrooms_Num`
# - `Amenity_Count` (count of amenities)
# - Dummy variables for the top 10 locations


# Handle missing values in Bedrooms_Num and Bathrooms_Num
print("Missing values before imputation:")
print(df[['Bedrooms_Num', 'Bathrooms_Num']].isnull().sum())

# Impute Bathrooms_Num with median (13 missing)
median_bathrooms = df['Bathrooms_Num'].median()
df['Bathrooms_Num'] = df['Bathrooms_Num'].fillna(median_bathrooms)

# Bedrooms_Num has no missing values

# Create dummies for top 10 locations
top_locs = df['Location'].value_counts().head(10).index
for loc in top_locs:
    df[f'Loc_{loc}'] = (df['Location'] == loc).astype(int)

# Define feature columns
feature_cols = ['Size_SQM', 'Bedrooms_Num', 'Bathrooms_Num', 'Amenity_Count'] + [f'Loc_{loc}' for loc in top_locs]
X = df[feature_cols].fillna(0)  # final safety net
y = df['Price_Millions']

print("Features shape:", X.shape)
print(X.head())


: 

In [ ]:
# ===================================================================================
#   4. Train/Test Split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")


: 

In [ ]:
# ===================================================================================
#   5. Train Linear Regression


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

: 

In [ ]:
# ===================================================================================
#   6. Evaluate the Model


# Train metrics
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
r2_train = r2_score(y_train, y_pred_train)

# Test metrics
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

print("Train MAE: {:.2f}M KSh".format(mae_train))
print("Train RMSE: {:.2f}M KSh".format(rmse_train))
print("Train R²: {:.3f}".format(r2_train))
print("\nTest MAE: {:.2f}M KSh".format(mae_test))
print("Test RMSE: {:.2f}M KSh".format(rmse_test))
print("Test R²: {:.3f}".format(r2_test))

# ===================================================================================
# **Interpretation:**  
# - MAE (Mean Absolute Error) tells  the average prediction error in millions of KSh. Our test MAE of about 8.4M means we're off by that much on average.  
# - R² of 0.68 means our model explains 68% of the price variance – a decent baseline.


: 

In [ ]:
# ===================================================================================
#   7. Feature Importance (Coefficients)


# Create a DataFrame of coefficients
coef_df = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': lr.coef_
}).sort_values('coefficient', ascending=False)

# Add the intercept separately
intercept_df = pd.DataFrame({'feature': ['Intercept'], 'coefficient': [lr.intercept_]})
coef_df = pd.concat([intercept_df, coef_df], ignore_index=True)

print(coef_df)

# Plot coefficients (excluding intercept for scale)
plt.figure(figsize=(10,8))
plot_df = coef_df[coef_df['feature'] != 'Intercept'].sort_values('coefficient', ascending=True)
plt.barh(plot_df['feature'], plot_df['coefficient'])
plt.xlabel('Coefficient (impact on price in M KSh)')
plt.title('Linear Regression Coefficients')
plt.tight_layout()
plt.savefig('../reports/coefficients.png')
plt.show()

# ===================================================================================
# **Observations:**  
# - Location dummies (e.g., `Loc_Kitusuru`) have large positive coefficients – location is the strongest driver.  
# - `Size_SQM` adds about 0.15M per sqm (150,000 KSh).  
# - `Amenity_Count` adds roughly 0.8M per additional amenity.


: 

In [ ]:
# ===================================================================================
#   8. Save Model 
# 
# save the model for later use (maybe in Streamlit- Day 5).


# Save the model using joblib as .pkl file - common format for scikit-learn models
import joblib

os.makedirs('../models', exist_ok=True)  # ensure models directory exists
joblib.dump(lr, '../models/linear_baseline.pkl')
print("Model saved to ../models/linear_baseline.pkl")

: 